In [1]:
import keras

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import os
import numpy as np
import cv2
import random
import Image

tf.logging.set_verbosity(tf.logging.INFO)

# emotion coding: 0=neutral, 1=anger, 2=contempt, 3=disgust, 4=fear, 5=happy, 6=sadness, 7=surprise
# e_labels = [0, 1, 2, 3, 4, 5, 6, 7]
# this is not in the domain of our loss function, so we will increase all values by 1

Using TensorFlow backend.


ModuleNotFoundError: No module named 'Image'

In [32]:
def get_tagged_images():
    emotions = []
    for path, name, files in os.walk('Emotion'):
        for file in files:
            if(file):
                with open(os.path.join(path, file)) as curf:
                    n = int(file[-14:-12])
                    
                    score = int(curf.read().strip(' ')[:1]) + 1
                    
                    # There are only labels in the dataset for the last image in a sequence, but the few before that are 
                    # still quite expressive. Including them will greatly augment our dataset. 
                    for i in range(4):
                        fi = file[:-14] + str(n - i).zfill(2) + "_emotion.txt"
                        emotions.append((os.path.join(path, file), fi[:-4], score))

    
    print(len(emotions), " tagged images found")
    return emotions

emotions = get_tagged_images()

1308  tagged images found


In [15]:
# want shape (num images, 115,600)

def get_image_data():
    
    data = []
    labels = []

    for i in range(len(emotions)):        
        im = os.path.join(os.path.join('aligned', emotions[i][1][:-8] + '.png'))
        
        # need to make the images three channels so that the pretrained model will work
  
        img = cv2.cvtColor(cv2.imread(im, 0), cv2.COLOR_GRAY2BGR)
        nimg = cv2.normalize(img.astype('float'), None, 0, 1,norm_type=cv2.NORM_MINMAX)

        data.append(nimg)
        labels.append(emotions[i][2])

    npdata = tf.stack(np.array(data, dtype=np.float32))
    nplabels = np.array(labels, dtype=np.int32)


    print(npdata.shape)
    npdata
    return npdata, nplabels
    
a, b = get_image_data()
print(a[0].shape)
print(a.shape)

(327, 340, 340, 3)
(340, 340, 3)
(327, 340, 340, 3)


In [16]:
print(a[0])
print(a.shape)

Tensor("strided_slice_3:0", shape=(340, 340, 3), dtype=float32)
(327, 340, 340, 3)


In [ ]:
m = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_tensor=a, input_shape=[340, 340, 3], pooling=max)

print(m.summary())

for layer in m.layers:
    layer.trainable = False
    
x = m.output
x = keras.layers.Dense(512, activation='relu')(x)

model = keras.models.Model(inputs=m.input, outputs = x)
print(model.summary())